In [13]:
import base64
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
from datetime import date, timedelta
from typing import Tuple, List, Union

In [2]:
def data_download() -> Tuple[pd.DataFrame]:
    conns_cols = ['id', 'installation_id',\
                'connected_at', 'is_internet_available',\
                'is_protected', 'captive_portal_mode',\
                'signal_strength', 'hotspot_id']

    hotspots_cols = ['id', 'foursquare_id',\
                    'google_place_id', 'created_at',\
                    'owner_id', 'score_v4', 'deleted_at']

    conns = pd.read_csv("data/conns_test.csv", usecols=conns_cols)
    hotspots = pd.read_csv("data/hotspots_test.csv", usecols=hotspots_cols)
    users = pd.read_csv("data/users_test.csv", usecols=['id', 'email'])

    conns['connected_at'] = conns['connected_at'].astype('datetime64[ns]').dt.normalize()

    hotspots['created_at'] = hotspots['created_at'].astype('datetime64[ns]').dt.normalize()
    hotspots['deleted_at'] = hotspots['deleted_at'].astype('datetime64[ns]').dt.normalize()

    users['user'] = users.email.apply(lambda x: x.split('_')[0])
    users.drop(columns='email', inplace=True)

    return conns, hotspots, users

In [3]:
def merge_datasets(conns: pd.DataFrame, hotspots: pd.DataFrame,
                    users: pd.DataFrame) -> Tuple[pd.DataFrame]:
    users_hotspots = users[['id', 'user']]\
                    .merge(hotspots, how='inner', left_on='id', right_on='owner_id')\
                    .drop(columns=['id_x', 'owner_id'])\
                    .rename(columns={'id_y': 'id'})

    users_conns = users_hotspots.merge(conns, left_on='id', right_on='hotspot_id')

    return users_hotspots, users_conns

In [4]:
conns, hotspots, users = data_download()

users_hotspots, users_conns = merge_datasets(conns, hotspots, users)

users_hotspots.head()

,user,id,foursquare_id,google_place_id,created_at,score_v4,deleted_at
0,1,16610538,NaN,ChIJc1MxT6I-3YkRWm-NUZwQ0fA,2022-08-09,0.78,NaT
1,1,16632145,4b59aedcf964a520999228e3,NaN,2022-08-24,0.85,NaT
2,1,15943091,40e0b100f964a520cd041fe3,NaN,2022-01-18,0.90,NaT
3,1,15290640,5b0780332f97ec002cd13f11,NaN,2021-08-18,0.78,NaT
4,1,16655963,4bc35661dce4eee1e98d719d,NaN,2022-09-10,0.86,NaT


Посчитать сколько wifi точек (мы wifi записи еще называем hotspots) создал пользователь. (owner_id - идентификатор(связь) пользователя id в таблице hotspots) 

In [5]:
def count_users_hotspots(users_hotspots: pd.DataFrame) -> pd.DataFrame:
    return users_hotspots\
            .groupby('user', as_index=False)\
            .agg({'id': 'count'})\
            .sort_values('id', ascending=False)\
            .rename(columns={'id': 'count'})

count_users_hotspots(users_hotspots).head(10)

,user,count
21,28,1392
12,2,605
80,81,541
35,40,406
18,25,401
3,11,391
37,42,307
26,32,300
32,38,269
70,72,269


In [18]:
def dataset_visualisation(dfs: Union[pd.DataFrame, list[pd.DataFrame]]) -> Union[str, list[str]]:
    if isinstance(dfs, pd.DataFrame):
        img = BytesIO()
        dfs.plot(kind='bar')
        plt.minorticks_on()
        plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
        plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
        plt.savefig(img, format='png')
        plt.close()
        img.seek(0)
        plot_url = base64.b64encode(img.getvalue()).decode('utf8')
    else:
        plot_url = list()
        for df in dfs:
            df.plot(kind='bar')
            plt.minorticks_on()
            plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
            plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
            plt.savefig(img, format='png')
            plt.close()
            img.seek(0)
            plot_url.append(base64.b64encode(img.getvalue()).decode('utf8'))
    return plot_url

dataset_visualisation(count_users_hotspots(users_hotspots).head(10))

'...'

Посчитать сколько hotspots у пользователя с привязкой к месту (если foursquare_id or google_place_id не NULL - место указано)

In [12]:
def count_users_hotspots_geo(users_hotspots: pd.DataFrame) -> pd.DataFrame:
    return users_hotspots[users_hotspots.foursquare_id.notnull() |\
            users_hotspots.google_place_id.notnull()]\
            .groupby('user', as_index=False)\
            .agg({'id': 'count'})\
            .sort_values('id', ascending=False)\
            .rename(columns={'id': 'count'})

count_users_hotspots_geo(users_hotspots)['count'].sum()

8291

Посчитать сколько hotspots пользователь создал за все время, за последний месяц, неделю

За все время:

In [15]:
def count_users_hotspots_over_time(users_hotspots: pd.DataFrame) -> List[pd.DataFrame]:

    result = []

    dates = users_hotspots.created_at.min(),\
            date.today() - pd.offsets.MonthBegin(1),\
            pd.to_datetime(date.today() - timedelta(days = date.today().weekday())),\
            pd.to_datetime(users_hotspots.created_at.max() -\
                timedelta(days = users_hotspots.created_at.max().weekday()))

    for dt in dates:
        result.append(users_hotspots\
            .loc[users_hotspots.created_at >= dt]\
            .groupby('user', as_index=False)\
            .agg({'id': 'count'})\
            .sort_values('id', ascending=False)\
            .rename(columns={'id': 'count'}))

    return result

count_users_hotspots_over_time(users_hotspots)[0]['count'].sum()

10236

In [12]:
users_hotspots.loc[users_hotspots.created_at >= users_hotspots.created_at.min()][
                "id"
            ].nunique()

10236

За последний месяц:

In [286]:
users_hotspots\
        .loc[(users_hotspots.created_at >= date.today() - pd.offsets.MonthBegin(1))]\
        .groupby('user', as_index=False)\
        .agg({'id': 'count'})\
        .sort_values('id', ascending=False)\
        .rename(columns={'id': 'count'})

,user,count
12,72,16
6,36,14
9,42,8
11,63,7
8,40,4
15,81,4
5,28,3
10,43,3
1,19,3
0,16,2


За последнюю неделю:

In [336]:
week_start = pd.to_datetime(date.today() - timedelta(days = date.today().weekday() % 6))

users_hotspots\
    .loc[users_hotspots.created_at >= week_start]\
    .groupby('user', as_index=False)\
    .agg({'id': 'count'})\
    .sort_values('id', ascending=False)\
    .rename(columns={'id': 'count'})

,user,count


Для проверки правильности посмотрим неделю от максимальной даты в датасете.

In [337]:
max_week_start = pd.to_datetime(users_hotspots.created_at.max() -\
                timedelta(days = users_hotspots.created_at.max().weekday() % 6))

users_hotspots\
    .loc[users_hotspots.created_at >= max_week_start]\
    .groupby('user', as_index=False)\
    .agg({'id': 'count'})\
    .sort_values('id', ascending=False)\
    .rename(columns={'id': 'count'})

,user,count
3,72,5
2,63,3
1,43,2
0,16,1
4,81,1
5,83,1


Посчитать сколько у пользователя хороших hotspots (score_v4 > 0.6), средних hotspots (0.3 < score_v4 < 0.6), плохих (score_v4  < 0.3). В ТЗ не указано в какие категории попадают граничные значения.

In [13]:
def count_users_hotspots_score(users_hotspots: pd.DataFrame) -> pd.DataFrame:
    users_hotspots['good_hs'] = np.where(users_hotspots.score_v4 > 0.6, 1, 0)
    users_hotspots['avg_hs'] = np.where((users_hotspots.score_v4 < 0.6)\
                                        & (users_hotspots.score_v4 > 0.3), 1, 0)
    users_hotspots['bad_hs'] = np.where(users_hotspots.score_v4 < 0.3, 1, 0)

    return users_hotspots\
            .groupby('user', as_index=False)\
            .agg({'good_hs': 'sum', 'avg_hs': 'sum', 'bad_hs': 'sum'})\
            .sort_values(by=['good_hs', 'avg_hs', 'bad_hs'], ascending=False)

count_users_hotspots_score(users_hotspots)['good_hs'].sum()

3620

Посчитать сколько у пользователя hotspots к которым было больше 1, 5 и 10 уникальных(!) подключений за все время, за последний год, за последний месяц, за последнюю неделю. (Уникальными считать подключения по полю installation_id)

3)В задании 1.5 уникальным считать первое найденное значение поля installation_id, все остальные такие же значения не уникальны?

1 пользователь может иметь 10 подключений к точке – это нужно считать как 1 подключение. Но учесть что там тренбо уникальны за день и т.д. то есть если пользователь каждый день по 5 раз подключается – считать как за 1 

Получается, что уникальным считаем только подключение к hotspots один раз в день. Несколько подключений от одного пользователя за день надо считать за 1

За все время:

In [19]:
def count_users_unique_hotspots(users_conns: pd.DataFrame,
                                users_hotspots: pd.DataFrame) -> List[pd.DataFrame]:

    result = []

    dates = users_hotspots.created_at.min(),\
            date.today() - pd.offsets.YearBegin(1),\
            date.today() - pd.offsets.MonthBegin(1),\
            pd.to_datetime(date.today() - timedelta(days = date.today().weekday())),\
            pd.to_datetime(users_hotspots.created_at.max() -\
                timedelta(days = users_hotspots.created_at.max().weekday()))    

    for dt in dates:
        users_conns_unique = users_conns\
            .loc[(users_conns.connected_at >= dt)]\
            .groupby(['user', 'connected_at', 'hotspot_id'], as_index=False)\
            .agg({'installation_id': 'count'})\
            .groupby(['user', 'hotspot_id'], as_index=False)\
            .agg({'connected_at': 'count'})

        users_conns_unique['more_1_conns'] = np.where(users_conns_unique.connected_at > 1, 1, 0)
        users_conns_unique['more_5_conns'] = np.where(users_conns_unique.connected_at > 5, 1, 0)
        users_conns_unique['more_10_conns'] = np.where(users_conns_unique.connected_at > 10, 1, 0)

        result.append(users_conns_unique\
            .groupby('user', as_index=False)\
            .agg({'more_1_conns': 'sum', 'more_5_conns': 'sum', 'more_10_conns': 'sum'})\
            .sort_values(by=['more_1_conns', 'more_5_conns', 'more_10_conns'], ascending=False))
    print(users_conns_unique)
    return result

count_users_unique_hotspots(users_conns, users_hotspots)[0]['more_1_conns'].sum()

    user  hotspot_id  connected_at  more_1_conns  more_5_conns  more_10_conns
0      1    14183671             2             1             0              0
1      1    16655963             1             0             0              0
2     10      905693             1             0             0              0
3     10    11347512             1             0             0              0
4     11     6838037             1             0             0              0
..   ...         ...           ...           ...           ...            ...
183   93    11599817             2             1             0              0
184   93    12121300             1             0             0              0
185   97     2670148             1             0             0              0
186   99    10805271             1             0             0              0
187   99    16340288             2             1             0              0

[188 rows x 6 columns]


1883

За последний год:

In [324]:
users_conns_unique_lastyear = users_conns\
    .loc[(users_conns.connected_at >= date.today() - pd.offsets.YearBegin(1))]\
    .groupby(['user', 'connected_at', 'hotspot_id'], as_index=False)\
    .agg({'installation_id': 'count'})\
    .groupby(['user', 'hotspot_id'], as_index=False)\
    .agg({'connected_at': 'count'})

users_conns_unique_lastyear['more_1_conns'] = np.where(users_conns_unique_lastyear.connected_at > 1, 1, 0)
users_conns_unique_lastyear['more_5_conns'] = np.where(users_conns_unique_lastyear.connected_at > 5, 1, 0)
users_conns_unique_lastyear['more_10_conns'] = np.where(users_conns_unique_lastyear.connected_at > 10, 1, 0)

users_conns_unique_lastyear\
    .groupby('user', as_index=False)\
    .agg({'more_1_conns': 'sum', 'more_5_conns': 'sum', 'more_10_conns': 'sum'})\
    .sort_values(by=['more_1_conns', 'more_5_conns', 'more_10_conns'], ascending=False)

,user,more_1_conns,more_5_conns,more_10_conns
21,28,251,126,69
70,72,140,102,84
12,2,136,64,39
80,81,85,32,15
3,11,70,42,24
...,...,...,...,...
55,59,1,0,0
56,6,1,0,0
62,65,1,0,0
72,74,1,0,0


За последний месяц:

In [325]:
users_conns_unique_lastmonth = users_conns\
    .loc[(users_conns.connected_at >= date.today() - pd.offsets.MonthBegin(1))]\
    .groupby(['user', 'connected_at', 'hotspot_id'], as_index=False)\
    .agg({'installation_id': 'count'})\
    .groupby(['user', 'hotspot_id'], as_index=False)\
    .agg({'connected_at': 'count'})

users_conns_unique_lastmonth['more_1_conns'] = np.where(users_conns_unique_lastmonth.connected_at > 1, 1, 0)
users_conns_unique_lastmonth['more_5_conns'] = np.where(users_conns_unique_lastmonth.connected_at > 5, 1, 0)
users_conns_unique_lastmonth['more_10_conns'] = np.where(users_conns_unique_lastmonth.connected_at > 10, 1, 0)

users_conns_unique_lastmonth\
    .groupby('user', as_index=False)\
    .agg({'more_1_conns': 'sum', 'more_5_conns': 'sum', 'more_10_conns': 'sum'})\
    .sort_values(by=['more_1_conns', 'more_5_conns', 'more_10_conns'], ascending=False)

,user,more_1_conns,more_5_conns,more_10_conns
20,28,23,3,1
55,72,20,9,3
11,2,15,2,0
64,82,9,1,0
27,36,7,2,0
...,...,...,...,...
44,55,0,0,0
47,62,0,0,0
59,77,0,0,0
70,90,0,0,0


За последнюю неделю:

In [328]:
users_conns_unique_lastweek = users_conns\
    .loc[(users_conns.connected_at >= pd.to_datetime(date.today()\
                    - timedelta(days = date.today().weekday() % 7)))]\
    .groupby(['user', 'connected_at', 'hotspot_id'], as_index=False)\
    .agg({'installation_id': 'count'})\
    .groupby(['user', 'hotspot_id'], as_index=False)\
    .agg({'connected_at': 'count'})

users_conns_unique_lastweek['more_1_conns'] = np.where(users_conns_unique_lastweek.connected_at > 1, 1, 0)
users_conns_unique_lastweek['more_5_conns'] = np.where(users_conns_unique_lastweek.connected_at > 5, 1, 0)
users_conns_unique_lastweek['more_10_conns'] = np.where(users_conns_unique_lastweek.connected_at > 10, 1, 0)

users_conns_unique_lastweek\
    .groupby('user', as_index=False)\
    .agg({'more_1_conns': 'sum', 'more_5_conns': 'sum', 'more_10_conns': 'sum'})\
    .sort_values(by=['more_1_conns', 'more_5_conns', 'more_10_conns'], ascending=False)

,user,more_1_conns,more_5_conns,more_10_conns


Для проверки используем последнюю неделю:

In [330]:
users_conns_unique_lastdateweek = users_conns\
    .loc[(users_conns.connected_at >= pd.to_datetime(hotspots.created_at.max() -\
                timedelta(days = hotspots.created_at.max().weekday() % 6)))]\
    .groupby(['user', 'connected_at', 'hotspot_id'], as_index=False)\
    .agg({'installation_id': 'count'})\
    .groupby(['user', 'hotspot_id'], as_index=False)\
    .agg({'connected_at': 'count'})

users_conns_unique_lastdateweek['more_1_conns'] = np.where(users_conns_unique_lastdateweek.connected_at > 1, 1, 0)
users_conns_unique_lastdateweek['more_5_conns'] = np.where(users_conns_unique_lastdateweek.connected_at > 5, 1, 0)
users_conns_unique_lastdateweek['more_10_conns'] = np.where(users_conns_unique_lastdateweek.connected_at > 10, 1, 0)

users_conns_unique_lastdateweek\
    .groupby('user', as_index=False)\
    .agg({'more_1_conns': 'sum', 'more_5_conns': 'sum', 'more_10_conns': 'sum'})\
    .sort_values(by=['more_1_conns', 'more_5_conns', 'more_10_conns'], ascending=False)

,user,more_1_conns,more_5_conns,more_10_conns
37,72,8,0,0
15,28,4,0,0
43,82,4,0,0
6,16,3,0,0
42,81,3,0,0
47,88,3,0,0
9,2,2,0,0
12,25,2,0,0
17,32,2,0,0
19,36,2,0,0
